### 1. Assignment Topic:

In this project, you will build a regression model using the Keras library to model the same data about concrete compressive strength that we used in labs 3.p In this course project, you will build a regression model using the deep learning Keras library, and then you will experiment with increasing the number of training epochs and changing number of hidden layers and you will see how changing these parameters impacts the performance of the model.

## Increasing the Number of Training Epochs - 2

In [1]:
import pandas as pd
import numpy as np

### 2. Concrete Data:

For your convenience, the data can be found here again: https://cocl.us/concrete_data. To recap, the predictors in the data of concrete strength include:

1-Cement

2-Blast Furnace Slag

3-Fly Ash

4-Water

5-Superplasticizer

6-Coarse Aggregate

7-Fine Aggregate

In [2]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

#### Let's check the dataset for any missing values. 

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

### Split data into predictors and target
The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

target = concrete_data["Strength"]
predictors = concrete_data.drop(["Strength", "Age"], axis=1)

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### Normalize predictors and target
The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569


In [10]:
n_cols = predictors_norm.shape[1]  # number of predictors
n_cols

7

### Importing Keras Libraries

In [11]:
import warnings
warnings.filterwarnings("ignore")
import keras

Using TensorFlow backend.


#### Importing the rest of the packages from the Keras library that we will need to build our regression model.

In [12]:
from keras.models import Sequential
from keras.layers import Dense

### Building Regression Model

In [13]:
# define regression model
def regression_model():
    # 1-Create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # 2-Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

#### The above function creates a model that has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.

Importing scikit-learn in order to randomly split the data into a training and test sets.

In [14]:
from sklearn.model_selection import train_test_split

#### We will train and test the model at the same time using the fit method. We will leave out 30% of the data for validation.

In [15]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

#### Train and Test the Network 

In [16]:
model = regression_model()

#### We will train the model for 120 epochs.  

In [17]:
epochs = 120
model.fit(X_train, y_train, epochs=epochs, verbose=1)



Epoch 1/120


2022-01-02 13:13:51.759824: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2022-01-02 13:13:51.766176: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2593905000 Hz
2022-01-02 13:13:51.767319: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56436ddba830 executing computations on platform Host. Devices:
2022-01-02 13:13:51.767361: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-01-02 13:13:51.874631: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

721/721 [==============================] - 0s 483us/step - loss: 7464.6784
Epoch 2/120
721/721 [==============================] - 0s 157us/step - loss: 1037.3098
Epoch 3/120
721/721 [==============================] - 0s 116us/step - loss: 384.8450
Epoch 4/120
721/721 [==============================] - 0s 136us/step - loss: 236.9303
Epoch 5/120
721/721 [==============================] - 0s 135us/step - loss: 189.4099
Epoch 6/120
721/721 [==============================] - 0s 90us/step - loss: 168.9335
Epoch 7/120
721/721 [==============================] - 0s 99us/step - loss: 162.9471
Epoch 8/120
721/721 [==============================] - 0s 85us/step - loss: 161.3206
Epoch 9/120
721/721 [==============================] - 0s 87us/step - loss: 158.3191
Epoch 10/120
721/721 [==============================] - 0s 104us/step - loss: 155.6560
Epoch 11/120
721/721 [==============================] - 0s 88us/step - loss: 154.3112
Epoch 12/120
721/721 [==============================] - 0s 88us/ste

In [18]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 88us/step


159.37911928047254

### Mean Squared Error
Let's import mean_squared_error from sklearn library to calculate our mean squared error.

In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(f"Mean: {mean}\nStandard Deviation: {standard_deviation}")

Mean: 159.37911685414875
Standard Deviation: 0.0
